New Project File:

Shell for the CAPTCHA Cracking Challenge

There are 8501 training images and a secret number of test images

In [3]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib import patches as ptc 
import math
import random
from PIL import Image
import os
import re

# Define function for converting text to one-hot numerical encoding

def OneHotConverter(textstring, dictionary): # Needs to be a reverse dictionary

    ans = np.zeros((6, 21))
    for i in range(6):
        char = textstring[i].upper()
        num = dictionary[char]#dict2
        ans[i][num] = 1
    return ans 


    


# Define dictionaries for encoding/decoding
dict1 = {0:'2', 1:'6', 2:'A', 3:'B', 4:'C', 5:'D', 6:'E', 7:'F', 8:'G', 9:'H', 10:'K', 11:'M', 
              12:'N', 13:'P', 14:'R', 15:'S', 16:'T', 17:'U', 18:'V', 19:'X', 20:'Z'}
dict2 = {v: k for k, v in dict1.items()}



trainfiles2 = os.listdir('./traindata/')
trainfiles = [f for f in trainfiles2 if os.path.isfile('.' +'/traindata/' + f)] #Filtering only the files.
valfiles2 = os.listdir('./valdata/')
valfiles = [f for f in valfiles2 if os.path.isfile('.' +'/valdata/' + f)]




random.shuffle(trainfiles)
numtrain = len(trainfiles)
numval = len(valfiles)

train_images = np.zeros((numtrain,250,50))
train_labels = np.zeros((numtrain,6,21))
train_labels1 = np.zeros((numtrain, 21))
train_labels2 = np.zeros((numtrain, 21))
train_labels3 = np.zeros((numtrain, 21))
train_labels4 = np.zeros((numtrain, 21))
train_labels5 = np.zeros((numtrain, 21))
train_labels6 = np.zeros((numtrain, 21))
train_numlist = np.zeros((numtrain))
train_letters = ['     '] * numtrain

val_images = np.zeros((numval,250,50))
val_labels = np.zeros((numval,6,21))
val_labels1 = np.zeros((numval, 21))
val_labels2 = np.zeros((numval, 21))
val_labels3 = np.zeros((numval, 21))
val_labels4 = np.zeros((numval, 21))
val_labels5 = np.zeros((numval, 21))
val_labels6 = np.zeros((numval, 21))
val_numlist = np.zeros((numval))
val_letters = ['     '] * numval


# Define training set
for i in range(numtrain):
    img = Image.open('./traindata/' + trainfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    train_images[i] = data
    train_numlist[i] = re.split(r"[_.]", trainfiles[i])[0]
    temp = re.split(r"[_.]", trainfiles[i])
    train_labels[i] = OneHotConverter(temp[1], dict2)
    train_labels1[i] = train_labels[i][0]
    train_labels2[i] = train_labels[i][1]
    train_labels3[i] = train_labels[i][2]
    train_labels4[i] = train_labels[i][3]
    train_labels5[i] = train_labels[i][4]
    train_labels6[i] = train_labels[i][5]
    train_letters[i] = temp[1].upper()

# Define validation set
for i in range(numval):
    img = Image.open('./valdata/' + valfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    val_images[i] = data
    val_numlist[i] = re.split(r"[_.]", valfiles[i])[0]
    temp = re.split(r"[_.]", valfiles[i])
    val_labels[i] = OneHotConverter(temp[1], dict2)
    val_labels1[i] = val_labels[i][0]
    val_labels2[i] = val_labels[i][1]
    val_labels3[i] = val_labels[i][2]
    val_labels4[i] = val_labels[i][3]
    val_labels5[i] = val_labels[i][4]
    val_labels6[i] = val_labels[i][5]
    val_letters[i] = temp[1].upper()



# Note to self: add validation and test sets, comment out test set construction since the students
# Won't get the test set until the final




# Below is an example of how you can get the info from a train or validation file
print(re.split(r"[_.]", trainfiles[0]))

print(train_letters[0])
print(train_numlist[0])



['5637', 'eszpbp', 'jpeg']
ESZPBP
5637.0


Define the model.  

Input layer: 250x50 pixel images in grayscale (0 to 1 values)

ADD NETWORK HERE


Final activation function needs to be softmax
Output: 6*21 matrix, each of the 6 21 entry rows is a confidence vector for the given character
Loss functions: 6 copies of cross-entropy for the 6 different characters


In [19]:


inputlayer = tf.keras.Input(shape=(250, 50, 1), name="Input")


# Right now the network is basically the same as our example McDigit one
# You'll need to add/change a lot between here and the dense layers at the end
convlayer1 = tf.keras.layers.Conv2D(filters=30, kernel_size=(4,4),padding='same', name="Convolution")(inputlayer)
maxpoollayer1 = tf.keras.layers.MaxPooling2D(pool_size=(4,4),strides=(2,2), name="MaxPool1")(convlayer1)


convlayer2 = tf.keras.layers.Conv2D(filters=20, kernel_size=(4,4),padding='same', name="Convolution2")(maxpoollayer1)
maxpoollayer2 = tf.keras.layers.MaxPooling2D(pool_size=(4,4),strides=(1,1), name="MaxPool2")(convlayer2)
dropout_layer1 = tf.keras.layers.Dropout(rate=0.25,name="Dropout1")(maxpoollayer2)

convlayer3 = tf.keras.layers.Conv2D(filters=10, kernel_size=(4,4),padding='same', name="Convolution3")(dropout_layer1)
maxpoollayer3 = tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(1,1), name="MaxPool3")(convlayer3)

flatlayer = tf.keras.layers.Flatten(name="Flatten")(maxpoollayer3)

# denselayer1 = tf.keras.layers.Dense(40, activation='relu', name='Dense')(flatlayer)







char1 = tf.keras.layers.Dense(21, activation='softmax', name='Char_1')(flatlayer)
char2 = tf.keras.layers.Dense(21, activation='softmax', name='Char_2')(flatlayer)
char3 = tf.keras.layers.Dense(21, activation='softmax', name='Char_3')(flatlayer)
char4 = tf.keras.layers.Dense(21, activation='softmax', name='Char_4')(flatlayer)
char5 = tf.keras.layers.Dense(21, activation='softmax', name='Char_5')(flatlayer)
char6 = tf.keras.layers.Dense(21, activation='softmax', name='Char_6')(flatlayer)

model = tf.keras.Model(inputs=inputlayer, outputs=[char1, char2, char3, char4, char5, char6])

model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(0.001),
  loss={
      "Char_1": tf.keras.losses.CategoricalCrossentropy(),
      "Char_2": tf.keras.losses.CategoricalCrossentropy(),
      "Char_3": tf.keras.losses.CategoricalCrossentropy(),
      "Char_4": tf.keras.losses.CategoricalCrossentropy(),
      "Char_5": tf.keras.losses.CategoricalCrossentropy(),
      "Char_6": tf.keras.losses.CategoricalCrossentropy()
      },
  metrics=['accuracy']
  )

tf.keras.utils.plot_model(model, "model.png")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 250, 50, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Convolution (Conv2D)           (None, 250, 50, 30)  510         ['Input[0][0]']                  
                                                                                                  
 MaxPool1 (MaxPooling2D)        (None, 124, 24, 30)  0           ['Convolution[0][0]']            
                                                                                                  
 Convolution2 (Conv2D)          (None, 124, 24, 20)  9620        ['MaxPool1[0][0]']         

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
batchsize = 128 # You may need to change this
numbatches = math.ceil(numtrain / batchsize)

checkpointfreq = 10 # Saves every this many epochs

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "myweights1",
    save_freq=numbatches * checkpointfreq,
    save_weights_only=True)


model.fit(train_images, [train_labels1, train_labels2, train_labels3,
                        train_labels4, train_labels5, train_labels6],
                        validation_data=(val_images, [
                        val_labels1, val_labels2, val_labels3,
                        val_labels4, val_labels5, val_labels6
                        ]),
                        batch_size=batchsize,
                        epochs=50,
                        callbacks=[model_checkpoint_callback])





Epoch 1/50
67/67 [==============================] - 36s 529ms/step - loss: 16.8818 - Char_1_loss: 2.7504 - Char_2_loss: 2.8399 - Char_3_loss: 2.8158 - Char_4_loss: 2.8680 - Char_5_loss: 2.8324 - Char_6_loss: 2.7753 - Char_1_accuracy: 0.1934 - Char_2_accuracy: 0.1601 - Char_3_accuracy: 0.1733 - Char_4_accuracy: 0.1595 - Char_5_accuracy: 0.1583 - Char_6_accuracy: 0.1832 - val_loss: 18.2088 - val_Char_1_loss: 2.8384 - val_Char_2_loss: 3.9424 - val_Char_3_loss: 2.7326 - val_Char_4_loss: 3.1355 - val_Char_5_loss: 2.8462 - val_Char_6_loss: 2.7136 - val_Char_1_accuracy: 0.2160 - val_Char_2_accuracy: 0.1320 - val_Char_3_accuracy: 0.2580 - val_Char_4_accuracy: 0.1820 - val_Char_5_accuracy: 0.1900 - val_Char_6_accuracy: 0.2900
Epoch 2/50
67/67 [==============================] - 37s 558ms/step - loss: 7.6933 - Char_1_loss: 1.1134 - Char_2_loss: 1.3567 - Char_3_loss: 1.3457 - Char_4_loss: 1.3964 - Char_5_loss: 1.3922 - Char_6_loss: 1.0890 - Char_1_accuracy: 0.6607 - Char_2_accuracy: 0.5802 - Char_

In [38]:
#model.save_weights('myweights1') 
#Use this to save weights.  Weights are saved after every 10 epochs

In [58]:
#model.load_weights('myweights1')
#Use this to load pre-trained weights or a checkpoint

Below here, put your function for taking the output of the model on a given image and returning a string.  Pass an image to the function.

For example, suppose we pass training image 0_vgxrub.jpeg to the model, and get an output of 6*21.  We pass this output to the function (along with the dictionary defined in the first cell), and it should return the string containing each of the six characters.  In this case, it would return VGXRUB.

In [9]:
# Example of how to pass a particular image to the model

test1 = np.expand_dims(train_images[0],0)
outtest1 = model.predict(test1)

# The first character's output, for example, is 

print(outtest1[0][0])
print(sum(outtest1[0][0]))

# Look at which entry has the largest value (probability).  




# Your function should go here:

def ConvertOutput(output, dictionary):

    string = ""
    max_val = 0
    for i in range(6):
        for j in range(21):
            if output[i][j]>max_val:
                max_val = output[i][j]
                index = j
        string += dictionary[index]

    return string



1/1 [==============================] - 0s 14ms/step
[5.8435871e-06 3.9144088e-06 4.2635801e-07 1.8845338e-06 2.2661762e-07
 4.3168336e-07 9.9979156e-01 3.7835235e-07 2.2315431e-12 9.6192482e-05
 2.0656225e-06 4.0300449e-07 5.7545538e-08 2.5928587e-06 3.1060892e-05
 7.2568067e-09 4.5032943e-05 6.9336329e-07 8.7256342e-07 3.4650895e-07
 1.6100599e-05]
1.0000000937396814


Below here is stuff for testing the model.  Don't run this code.  Leave it commented out for now.

In [11]:
"""testfiles2 = os.listdir('./testdata/')
testfiles = [f for f in testfiles2 if os.path.isfile('.' +'/testdata/' + f)]

numtest = len(testfiles)

test_images = np.zeros((numtest,250,50))
test_labels = np.zeros((numtest,6,21))
test_labels1 = np.zeros((numtest, 21))
test_labels2 = np.zeros((numtest, 21))
test_labels3 = np.zeros((numtest, 21))
test_labels4 = np.zeros((numtest, 21))
test_labels5 = np.zeros((numtest, 21))
test_labels6 = np.zeros((numtest, 21))
test_numlist = np.zeros((numtest))
test_letters = ['     '] * numtest



for i in range(numval):
    img = Image.open('./testdata/' + testfiles[i]).convert('L')
    data = np.transpose(np.asarray(img)) / 255
    test_images[i] = data
    test_numlist[i] = re.split(r"[_.]", testfiles[i])[0]
    temp = re.split(r"[_.]", testfiles[i])
    test_labels[i] = OneHotConverter(temp[1], dict2)
    test_labels1[i] = test_labels[i][0]
    test_labels2[i] = test_labels[i][1]
    test_labels3[i] = test_labels[i][2]
    test_labels4[i] = test_labels[i][3]
    test_labels5[i] = test_labels[i][4]
    test_labels6[i] = test_labels[i][5]
    test_letters[i] = temp[1].upper()


                   """

In [12]:
""" results = model.evaluate(test_images, [test_labels1, test_labels2, test_labels3,
                        test_labels4, test_labels5, test_labels6])
avgaccuracy = 100*sum(results[-6:])/6 


print(f"Average character accuracy on test set: {avgaccuracy:.2f} percent") """

/Users/safal/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/utils/generic_utils.py:238: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer